In [12]:
from gensim.models import Word2Vec

In [13]:
print("K-Means")

w2v_model = Word2Vec.load('w2v.model')

vocabulary = []
for word in w2v_model.wv.key_to_index:
    vocabulary.append(word)
print(len(vocabulary))

K-Means
34223


In [14]:
import pre_processing
import nltk

In [16]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/trannguyenhan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [28]:
content = """
Trump cáo buộc đảng Dân chủ đang theo đuổi "nỗ lực đảo chính bất hợp pháp, mang tính đảng phái" khi tìm cách luận tội ông.

"Chính các người mới là phe can thiệp vào bầu cử Mỹ, phá vỡ nền dân chủ của đất nước và cản trở công lý. Điều này không là gì khác ngoài một nỗ lực đảo chính bất hợp pháp, mang tính đảng phái và sẽ thất bại nặng nề tại hòm phiếu, dựa trên tình hình gần đây", Tổng thống Mỹ Donald Trump viết trong lá thư dài 6 trang gửi Chủ tịch Hạ viện Nancy Pelosi vào hôm qua để chỉ trích cuộc điều tra luận tội của phe Dân chủ. 

Ông chủ Nhà Trắng viết thêm rằng bà Pelosi "đang biến hạ viện từ cơ quan lập pháp được kính trọng thành một phòng xử án của cuộc đàn áp đảng phái", đồng thời "gần như không thèm che giấu sự thù ghét" đối với ông. 

"Bằng cách thúc đẩy cuộc luận tội trái phép đó, các người đang vi phạm lời thề nhậm chức, phá vỡ lòng trung thành với hiến pháp và tuyên chiến với nền dân chủ Mỹ. Các người coi dân chủ là kẻ thù của chính mình", Trump viết.
Bức thư của Trump được gửi đi trong bối cảnh hạ viện, nơi đảng Dân chủ chiếm đa số, dự kiến bỏ phiếu vào hôm nay về các điều khoản luận tội Tổng thống Mỹ và khả năng cao sẽ thông qua để đưa Trump ra xét xử tại thượng viện. Ông gần như chắc chắn sẽ trở thành tổng thống Mỹ thứ ba trong lịch sử bị luận tội. 

Trước đó, Ủy ban Tư pháp Hạ viện đã thông qua hai điều khoản luận tội Trump, bao gồm lạm dụng quyền lực và cản trở quốc hội. Phe Dân chủ cáo buộc Trump thúc ép Ukraine điều tra cha con cựu phó tổng thống Mỹ Joe Biden, đối thủ của Trump trong cuộc bầu cử năm 2020, gây nguy hiểm cho hiến pháp Mỹ, an ninh quốc gia và tính công bằng của bầu cử. 

Trong khi đó, đảng Cộng hòa vẫn cố gắng bảo vệ Trump, khi lãnh đạo phe đa số tại thượng viện Mitch McConnell bác bỏ yêu cầu của đảng Dân chủ nhằm đề nghị 4 quan chức cũ và đương nhiệm ra làm chứng tại phiên tòa luận tội dự kiến diễn ra ở thượng viện vào tháng tới. 

Hạ viện Mỹ mở cuộc điều tra luận tội Trump từ tháng 9, sau khi một người tố giác giấu tên đệ đơn cáo buộc Tổng thống Mỹ dùng gói viện trợ quân sự để gây áp lực lên người đồng cấp Ukraine Volodymyr Zelensky. Tuy nhiên, Trump phủ nhận các cáo buộc và gọi cuộc điều tra luận tội là trò lừa bịp.
"""

In [33]:
import numpy as np 
import nltk
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

n_clusters = 3
def k_means(sentences):
    # read root sentences and normalize sentences
    root_sentences = nltk.sent_tokenize(content)
    contents_parsed = pre_processing.text_preprocess(content)
    sentences = nltk.sent_tokenize(contents_parsed)

    X = []
    for sentence in sentences:
        words = sentence.split(" ")
        sentence_vec = np.zeros((128))
        
        for word in words:
            if word in vocabulary:
                sentence_vec+=w2v_model.wv[word]

        X.append(sentence_vec)

    print("Number of element: " + str(len(X)))

    kmeans = KMeans(n_clusters=n_clusters)
    kmeans = kmeans.fit(X)

    print("K-Means Clustering\n")
    avg = []
    for j in range(n_clusters):
        print("Cụm", j+1)
        idx = np.where(kmeans.labels_ == j)[0]
        print(idx)
        avg.append(np.mean(idx))
        print("Thứ tự trung bình: ", round(np.mean(idx), 2))
        print("="*115)

    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, X)

    print("Các câu gần", n_clusters, "tâm cụm nhất",closest)
    ordering = sorted(range(n_clusters), key=lambda k: avg[k])

    print("\nKết quả tóm tắt:\n")
    
    # summary = ' '.join(["(Câu " + str(closest[idx]) + ") " + sentences[closest[idx]] for idx in ordering])
    root_summary = ' '.join([root_sentences[closest[idx]] for idx in ordering])
    print(root_summary)

k_means(content)

Number of element: 13
K-Means Clustering

Cụm 1
[ 0  1  4  5  7  8 12]
Thứ tự trung bình:  5.29
Cụm 2
[ 2  6  9 10 11]
Thứ tự trung bình:  7.6
Cụm 3
[3]
Thứ tự trung bình:  3.0
Các câu gần 3 tâm cụm nhất [ 8 10  3]

Kết quả tóm tắt:

Ông chủ Nhà Trắng viết thêm rằng bà Pelosi "đang biến hạ viện từ cơ quan lập pháp được kính trọng thành một phòng xử án của cuộc đàn áp đảng phái", đồng thời "gần như không thèm che giấu sự thù ghét" đối với ông. Trước đó, Ủy ban Tư pháp Hạ viện đã thông qua hai điều khoản luận tội Trump, bao gồm lạm dụng quyền lực và cản trở quốc hội. Trong khi đó, đảng Cộng hòa vẫn cố gắng bảo vệ Trump, khi lãnh đạo phe đa số tại thượng viện Mitch McConnell bác bỏ yêu cầu của đảng Dân chủ nhằm đề nghị 4 quan chức cũ và đương nhiệm ra làm chứng tại phiên tòa luận tội dự kiến diễn ra ở thượng viện vào tháng tới.
